In [1]:
import pickle
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import types
from scipy.stats import mannwhitneyu
import statsmodels.api as sm
import pylab as pl
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA
from decimal import *
getcontext().prec = 30
import math

In [2]:
with open("/home/allan/Desktop/enron_fraud_detection/final_project_dataset.pkl","r") as f:
    data_dict = pickle.load(f)
data_dict.pop('TOTAL')
data_dict.pop('THE TRAVEL AGENCY IN THE PARK')

column_names =["name"]+[category for category in data_dict.values()[0].keys()]
enron = pd.DataFrame(columns=column_names)

i=0
for name,values in data_dict.items():
    datum_list={'name':name}
    for key,value in values.items():
        if value=="NaN":
            datum_list[key]=np.nan
        elif not isinstance(value,basestring):
            datum_list[key]=float(value)
        else:
            datum_list[key]=value
    enron=enron.append(pd.DataFrame(data=datum_list,index=[i]))
    i+=1

In [33]:
#get rid of stuff and redo analysis...
#enron["-deferred_income"]=-enron["deferred_income"]
#enron["-deferral_payments"]=-enron["deferral_payments"]
#enron["rel_total_payments"]=enron["total_payments"]/enron["total_stock_value"]
enron["rel_eso"]=enron["exercised_stock_options"]/enron["total_stock_value"]
#enron["percent_from_poi"] = enron["from_this_person_to_poi"] / enron["from_messages"]
#enron["percent_to_poi"] = enron["from_poi_to_this_person"] / enron["to_messages"]
#enron["percent_shared_poi"] = enron["shared_receipt_with_poi"] / enron["to_messages"]
#enron["percent_deferred"] = enron["deferral_payments"] / enron["deferred_income"]
#enron["percent_bonus"] = enron["bonus"] / enron["salary"]
#enron["percent_stock"] = enron["exercised_stock_options"] / enron["total_stock_value"]
##enron = enron.drop(labels=['deferral_payments','deferred_income','total_payments','from_this_person_to_poi','from_poi_to_this_person','from_messages','to_messages','shared_receipt_with_poi','director_fees','restricted_stock_deferred','loan_advances','total_stock_value','other','restricted_stock'],axis=1)

In [34]:
enron_no_outliers=enron.copy()
enron_outliers=enron.copy()

for column in enron.loc[:,(enron.dtypes == np.float64)]:
    outliers = np.abs(enron[column])>np.abs(enron[column]).quantile(0.9)
    mean_no_outliers = enron[~outliers][column].mean()
    enron_no_outliers.loc[outliers,(column)]=mean_no_outliers
    enron_outliers.loc[~outliers,(column)]=0

enron_no_norm = enron_no_outliers.copy()
for column in enron.loc[:,(enron.dtypes == np.float64)]:
    col = enron_no_norm[column]
    enron_no_norm.loc[:,(column)] = enron_no_norm[column].fillna(col.mean(axis=0))
    enron_no_norm.loc[:,(column)] = (col-col.min(axis=0))/col.mean(axis=0)

for column in enron.loc[:,(enron.dtypes == np.float64)]:
    col = enron_outliers[column]
    enron_outliers.loc[:,(column)] = enron_outliers[column].fillna(col.mean(axis=0))
    enron_outliers.loc[:,(column)] = (col-col.min(axis=0))/col.mean(axis=0)


In [8]:
for npoi in enron_no_norm[enron['poi']==0]['name']:
    print '***\n', npoi
    for column in enron.loc[:,(enron.dtypes == np.float64) & (enron.columns!='poi')]:
        bottom_tenth = np.percentile(enron_no_norm[column].fillna(0),5)
        top_tenth = np.percentile(enron_no_norm[column].fillna(0),90)
        idx2 = abs(enron_no_norm[column].fillna(0))<abs(bottom_tenth)
        idx1 = abs(enron_no_norm[column])>abs(top_tenth)
        tenth=enron_no_norm[idx1]['name'].tolist()
        if npoi in tenth:
            print '\t',column,":\t",enron[enron['name']==npoi][column].tolist()

***
METTS MARK
	director_fees :	[nan]
	expenses :	[94299.0]
	salary :	[365788.0]
***
BAXTER JOHN C
	bonus :	[1200000.0]
	deferral_payments :	[1295738.0]
	director_fees :	[nan]
	exercised_stock_options :	[6680544.0]
	long_term_incentive :	[1586055.0]
***
ELLIOTT STEVEN
	director_fees :	[nan]
	exercised_stock_options :	[4890344.0]
	expenses :	[78552.0]
	restricted_stock :	[1788391.0]
	total_stock_value :	[6678735.0]
***
CORDES WILLIAM R
	director_fees :	[nan]
***
MORDAUNT KRISTINA M
	director_fees :	[nan]
***
MEYER ROCKFORD G
	deferral_payments :	[1848227.0]
	director_fees :	[nan]
***
MCMAHON JEFFREY
	bonus :	[2600000.0]
	director_fees :	[nan]
	from_this_person_to_poi :	[26.0]
	long_term_incentive :	[694862.0]
	salary :	[370448.0]
	shared_receipt_with_poi :	[2228.0]
***
HORTON STANLEY C
	director_fees :	[nan]
	exercised_stock_options :	[5210569.0]
	from_messages :	[1073.0]
	restricted_stock :	[2046079.0]
	total_payments :	[3131860.0]
	total_stock_value :	[7256648.0]
***
PIPER GREGORY F
	

In [20]:
q = enron['name']
q.tolist()

['METTS MARK',
 'BAXTER JOHN C',
 'ELLIOTT STEVEN',
 'CORDES WILLIAM R',
 'HANNON KEVIN P',
 'MORDAUNT KRISTINA M',
 'MEYER ROCKFORD G',
 'MCMAHON JEFFREY',
 'HORTON STANLEY C',
 'PIPER GREGORY F',
 'HUMPHREY GENE E',
 'UMANOFF ADAM S',
 'BLACHMAN JEREMY M',
 'SUNDE MARTIN',
 'GIBBS DANA R',
 'LOWRY CHARLES P',
 'COLWELL WESLEY',
 'MULLER MARK S',
 'JACKSON CHARLENE R',
 'WESTFAHL RICHARD K',
 'WALTERS GARETH W',
 'WALLS JR ROBERT H',
 'KITCHEN LOUISE',
 'CHAN RONNIE',
 'BELFER ROBERT',
 'SHANKMAN JEFFREY A',
 'WODRASKA JOHN',
 'BERGSIEKER RICHARD P',
 'URQUHART JOHN A',
 'BIBI PHILIPPE A',
 'RIEKER PAULA H',
 'WHALEY DAVID A',
 'BECK SALLY W',
 'HAUG DAVID L',
 'ECHOLS JOHN B',
 'MENDELSOHN JOHN',
 'HICKERSON GARY J',
 'CLINE KENNETH W',
 'LEWIS RICHARD',
 'HAYES ROBERT E',
 'MCCARTY DANNY J',
 'KOPPER MICHAEL J',
 'LEFF DANIEL P',
 'LAVORATO JOHN J',
 'BERBERIAN DAVID',
 'DETMERING TIMOTHY J',
 'WAKEHAM JOHN',
 'POWERS WILLIAM',
 'GOLD JOSEPH',
 'BANNANTINE JAMES M',
 'DUNCAN JOHN H'

In [44]:
for column in enron.loc[:,(enron.dtypes == float) & (enron.columns != "poi")].columns:
    u,p=mannwhitneyu(enron_no_norm[column][enron["poi"]==True],enron_no_norm[column][enron["poi"]!=True])
    print column,':',p

bonus : 0.00274005053925
deferral_payments : 0.159139403959
deferred_income : 0.194312837404
director_fees : 0.264773478688
exercised_stock_options : 0.084448844992
expenses : 0.442336551355
from_messages : 0.115702104713
from_poi_to_this_person : 0.000986109957336
from_this_person_to_poi : 0.017467876044
loan_advances : 0.49405611085
long_term_incentive : 0.112741401166
other : 0.106993883084
restricted_stock : 0.0286588210295
restricted_stock_deferred : 0.381351511715
salary : 0.0652413870301
shared_receipt_with_poi : 0.000742417620693
to_messages : 0.0973725000552
total_payments : 0.0262050991902
total_stock_value : 0.162779718384
rel_eso : 0.361953983175


In [15]:
for column in enron:
    print sum(pd.notnull(enron[column]))-sum(pd.notnull(enron[column])&enron["poi"]==True),"::",sum(pd.notnull(enron[column])&enron["poi"]==True),'\t',column

65 :: 16 	bonus
33 :: 5 	deferral_payments
37 :: 11 	deferred_income
16 :: 0 	director_fees
93 :: 18 	email_address
89 :: 12 	exercised_stock_options
76 :: 18 	expenses
72 :: 14 	from_messages
72 :: 14 	from_poi_to_this_person
72 :: 14 	from_this_person_to_poi
2 :: 1 	loan_advances
53 :: 12 	long_term_incentive
126 :: 18 	name
73 :: 18 	other
126 :: 18 	poi
92 :: 17 	restricted_stock
17 :: 0 	restricted_stock_deferred
77 :: 17 	salary
72 :: 14 	shared_receipt_with_poi
72 :: 14 	to_messages
105 :: 18 	total_payments
107 :: 18 	total_stock_value


In [304]:
print enron.corr()

                            bonus  exercised_stock_options  expenses  \
bonus                    1.000000                 0.506998  0.033799   
exercised_stock_options  0.506998                 1.000000  0.019412   
expenses                 0.033799                 0.019412  1.000000   
long_term_incentive      0.625259                 0.500016 -0.100766   
poi                      0.302384                 0.503551  0.060292   
salary                   0.523190                 0.607324  0.145364   
percent_to_poi          -0.061878                 0.020597 -0.118860   
percent_shared_poi      -0.046763                 0.070133  0.197029   
percent_deferred         0.224723                 0.210155  0.000701   

                         long_term_incentive       poi    salary  \
bonus                               0.625259  0.302384  0.523190   
exercised_stock_options             0.500016  0.503551  0.607324   
expenses                           -0.100766  0.060292  0.145364   
long_te

In [48]:
print enron.columns

Index([u'bonus', u'deferral_payments', u'deferred_income', u'director_fees',
       u'email_address', u'exercised_stock_options', u'expenses',
       u'from_messages', u'from_poi_to_this_person',
       u'from_this_person_to_poi', u'loan_advances', u'long_term_incentive',
       u'name', u'other', u'poi', u'restricted_stock',
       u'restricted_stock_deferred', u'salary', u'shared_receipt_with_poi',
       u'to_messages', u'total_payments', u'total_stock_value'],
      dtype='object')


In [54]:
columns=[
    #u'total_stock_value',
    u'total_payments',
    #u'from_poi_to_this_person',
    #u'from_this_person_to_poi',
    #u'shared_receipt_with_poi',
    u'expenses',
    u'exercised_stock_options',
    u'other',    
    u'deferred_income',
    u'deferral_payments',
    u'bonus',
    u'long_term_incentive',
    u'restricted_stock',
    #u'salary',
    u'email_feature'
]

email_columns=[
    u'from_poi_to_this_person',
    u'from_this_person_to_poi',
    u'shared_receipt_with_poi',
    ]

pca1_columns=[
    "total_stock_value",
    "exercised_stock_options",
    "rel_eso"
]

In [55]:
pca = PCA(n_components=6)
qqq=enron[columns].fillna(enron[columns].mean())
values = (qqq - qqq.mean()) / (qqq.max() - qqq.min())
decomp = pca.fit_transform(values) 
for i in range(len(pca.components_)):
    for j in range(len(columns)):
        print columns[j],pca.components_[i][j]
    print '***'

total_payments -0.298933246181
expenses -0.0299122661291
exercised_stock_options -0.460469030331
other -0.375999267808
deferred_income 0.330946180927
deferral_payments -0.143231018247
bonus -0.434477882116
long_term_incentive -0.325489312696
restricted_stock -0.352112284987
email_feature 0.0871728750933
***
total_payments 0.128791674367
expenses 0.849127583287
exercised_stock_options 0.0739910026414
other 0.0779514426113
deferred_income 0.453832077753
deferral_payments -0.171205380972
bonus 0.0193793391596
long_term_incentive -0.0160265870772
restricted_stock 0.120338785235
email_feature -0.0213374144225
***
total_payments 0.151207611492
expenses -0.50911484392
exercised_stock_options 0.183213063089
other 0.0958883459574
deferred_income 0.686805142732
deferral_payments -0.349640031509
bonus 0.0336845640528
long_term_incentive 0.155636769255
restricted_stock 0.204534140048
email_feature 0.118471973855
***
total_payments 0.00809541473842
expenses -0.0446772681365
exercised_stock_options 

In [49]:
pca = PCA(n_components=3)
qqq=enron[email_columns].fillna(enron[email_columns].mean())
values = (qqq - qqq.mean()) / (qqq.max() - qqq.min())
decomp = pca.fit_transform(values) 
for i in range(len(pca.components_)):
    for j in range(len(email_columns)):
        print columns[j],pca.components_[i][j]
    print '***'
enron["email_feature"] = decomp[:,1]

total_stock_value 0.511968746963
total_payments 0.424071344557
expenses 0.747028444478
***
total_stock_value -0.162941153689
total_payments 0.901798952662
expenses -0.400260951645
***
total_stock_value 0.843408668777
total_payments -0.0831994212901
expenses -0.530791553936
***


In [41]:
pca = PCA(n_components=3)
qqq=enron[pca1_columns].fillna(enron[pca1_columns].mean())
values = (qqq - qqq.mean()) / (qqq.max() - qqq.min())
decomp = pca.fit_transform(values) 
for i in range(len(pca.components_)):
    for j in range(len(pca1_columns)):
        print pca1_columns[j],pca.components_[i][j]
    print '***'

total_stock_value -0.202201236612
exercised_stock_options -0.294961652969
rel_eso -0.933869521502
***
total_stock_value -0.652478421525
exercised_stock_options -0.670536141269
rel_eso 0.353062590339
***
total_stock_value -0.730333190644
exercised_stock_options 0.680719403668
rel_eso -0.0568728767822
***


In [222]:
ax=enron[enron_norm["poi"]==True].plot(kind='scatter',x='exercised_stock_options',y='total_stock_value',color="darkblue")
enron[enron_norm["poi"]==False].plot(kind='scatter',x='exercised_stock_options',y='total_stock_value',color="darkgreen",ax=ax)
plt.show()
ax=enron[enron_norm["poi"]==True].plot(kind='scatter',x='total_payments',y='total_stock_value',color="darkblue")
enron[enron_norm["poi"]==False].plot(kind='scatter',x='total_payments',y='total_stock_value',color="darkgreen",ax=ax)
plt.show()

TypeError: Empty 'DataFrame': no numeric data to plot

In [51]:
#td = plt.figure().gca(projection='3d')
#td.scatter(enron_norm[enron["poi"]==True]['long_term_incentive'] , enron_norm[enron["poi"]==True]['total_stock_value'] ,enron_norm[enron["poi"]==True]['bonus'],color='darkblue')
#td.scatter(enron_norm[enron["poi"]==False]['long_term_incentive'] , enron_norm[enron["poi"]==False]['total_stock_value'] ,enron_norm[enron["poi"]==False]['bonus'],color='darkgreen')
#plt.show()
#td = plt.figure().gca(projection='3d')
#td.scatter(enron_norm[enron["poi"]==True]['exercised_stock_options'] , enron_norm[enron["poi"]==True]['total_stock_value'] ,enron_norm[enron["poi"]==True]['total_payments'],color='darkblue')
#td.scatter(enron_norm[enron["poi"]==False]['exercised_stock_options'] , enron_norm[enron["poi"]==False]['total_stock_value'] ,enron_norm[enron["poi"]==False]['total_payments'],color='darkgreen')
#plt.show()


In [43]:

pca1=decomp[:,0]
pca2=decomp[:,1]
pca3=decomp[:,2]
#pca4=decomp[:,3]
#pca5=decomp[:,4]
#pca6=decomp[:,5]
#pca7=decomp[:,6]

poi_ind=(enron["poi"]==True).nonzero()
non_poi_ind=(enron["poi"]==False).nonzero()
color_array = np.array(["blue"]*len(pca1))
color_array[poi_ind]='red'

#plt.figure()
#plt.scatter(pca1,pca2,color=color_array)
#plt.figure()
#plt.scatter(pca3,pca2,color=color_array)
#plt.figure()
#plt.scatter(pca1,pca3,color=color_array)
#plt.plot(pca1[non_poi_ind],pca2[non_poi_ind],'bo')
#plt.show()

#print 
td = plt.figure().gca(projection='3d')
td.scatter(pca1, enron["exercised_stock_options"] , enron["exercised_stock_options"] ,color=color_array)
#td.scatter(pca1[non_poi_ind], pca2[non_poi_ind] , pca3[non_poi_ind],color='blue')
plt.show()

In [44]:
print pca3[poi_ind]

[ 1.11973386  1.26162877  0.23353535 -1.20171156  0.29600907 -0.81380942
 -1.32909018  0.17603235  2.19707111  0.29938751  0.82305675  0.80254159
  0.78701765 -0.15143608 -0.65765815 -1.39367682  0.24949897 -1.35145088]


In [53]:
len(df[df["poi"]==0])

126

In [136]:
cond = df[df["deferred_income"].notnull()]["deferred_income"]<-2000000
df[df["deferred_income"].notnull()][cond][["name","poi"]]

,name,poi
4,HANNON KEVIN P,1
82,BELDEN TIMOTHY N,1
88,RICE KENNETH D,1
107,ALLEN PHILLIP K,0
127,FREVERT MARK A,0


In [142]:
cond = df[df["expenses"].notnull()]["expenses"]>100000
df[df["expenses"].notnull()][cond][["name","poi"]]

,name,poi
7,MCMAHON JEFFREY,0
25,SHANKMAN JEFFREY A,0
28,URQUHART JOHN A,0
41,KOPPER MICHAEL J,1
46,WAKEHAM JOHN,0
51,SHAPIRO RICHARD S,0
56,KISHKILL JOSEPH G,0
71,MCCLELLAN GEORGE,0
108,SHARP VICTORIA T,0
129,BAY FRANKLIN R,0


In [153]:
cond = df[df["exercised_stock_options"].notnull()]["exercised_stock_options"]>5000000
df[df["exercised_stock_options"].notnull()][cond][["name","poi"]]

,name,poi
1,BAXTER JOHN C,0
4,HANNON KEVIN P,1
8,HORTON STANLEY C,0
65,LAY KENNETH L,1
80,REDMOND BRIAN L,0
88,RICE KENNETH D,1
92,OVERDYKE JR JERE C,0
95,SKILLING JEFFREY K,1
106,CHRISTODOULOU DIOMEDES,0
115,DIMICHELE RICHARD G,0


In [14]:
cond = enron[enron["long_term_incentive"].notnull()]["long_term_incentive"]>1000000
enron[enron["long_term_incentive"].notnull()][cond][["name","poi"]]

,name,poi
1,BAXTER JOHN C,0
4,HANNON KEVIN P,1
17,MULLER MARK S,0
34,ECHOLS JOHN B,0
42,LEFF DANIEL P,0
43,LAVORATO JOHN J,0
60,DELAINEY DAVID W,1
65,LAY KENNETH L,1
83,DURAN WILLIAM D,0
85,FASTOW ANDREW S,1


In [156]:
cond = df[df["bonus"].notnull()]["exercised_stock_options"]>5000000
df[df["bonus"].notnull()][cond][["name","poi"]]

,name,poi
1,BAXTER JOHN C,0
4,HANNON KEVIN P,1
65,LAY KENNETH L,1
88,RICE KENNETH D,1
95,SKILLING JEFFREY K,1
115,DIMICHELE RICHARD G,0
126,DERRICK JR. JAMES V,0
127,FREVERT MARK A,0
128,PAI LOU L,0


In [202]:
cond = df[df["salary"].notnull()]["exercised_stock_options"]>4000000
df[df["salary"].notnull()][cond][["name","poi"]]

NameError: name 'df' is not defined

In [204]:
cond = enron_norm[enron_norm["percent_from_poi"].notnull()]["percent_from_poi"]>2
enron_norm[enron_norm["percent_from_poi"].notnull()][cond][["name","poi"]]

,name,poi
4,HANNON KEVIN P,2.636549
10,HUMPHREY GENE E,-0.376650
97,SHERRICK JEFFREY B,-0.376650
121,GARLAND C KEVIN,-0.376650


In [205]:
cond = enron_norm[enron_norm["percent_to_poi"].notnull()]["percent_to_poi"]>2
enron_norm[enron_norm["percent_to_poi"].notnull()][cond][["name","poi"]]

,name,poi
16,COLWELL WESLEY,2.636549
55,DEFFNER JOSEPH M,-0.376650
80,REDMOND BRIAN L,-0.376650
142,DONAHUE JR JEFFREY M,-0.376650


In [210]:
cond = enron_norm[enron_norm["percent_sared_poi"].notnull()]["percent_sared_poi"]>1
enron_norm[enron_norm["percent_sared_poi"].notnull()][cond][["name","poi"]]

,name,poi
4,HANNON KEVIN P,2.636549
7,MCMAHON JEFFREY,-0.376650
10,HUMPHREY GENE E,-0.376650
12,BLACHMAN JEREMY M,-0.376650
13,SUNDE MARTIN,-0.376650
30,RIEKER PAULA H,2.636549
42,LEFF DANIEL P,-0.376650
88,RICE KENNETH D,2.636549
97,SHERRICK JEFFREY B,-0.376650
127,FREVERT MARK A,-0.376650
